In [0]:
%load_ext autoreload
%autoreload 2
# Enables autoreload; learn more at https://docs.databricks.com/en/files/workspace-modules.html#autoreload-for-python-modules
# To disable autoreload; run %autoreload 0

In [0]:
!pip install -r requirements.txt
dbutils.library.restartPython()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/60.4 kB ? eta -:--:--
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/68.0 kB ? eta -:--:--
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.0/68.0 kB 4.8 MB/s eta 0:00:00

*** WARNING: max output size exceeded, skipping output. ***

m 15.6/37.8 MB 109.9 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━ 19.4/37.8 MB 110.0 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━ 23.3/37.8 MB 111.6 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━ 27.5/37.8 MB 115.5 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━━━━━ 31.4/37.8 MB 114.6 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━ 34.9/37.8 MB 108.8 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 37.8/37.8 MB 111.2 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 37.8/37.8 MB 111.2 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [0]:
from constants import * 
import networkx as nx
import pyvis

from src.helper_functions import *
from src.llama_index_graph_rag_extractor import GraphRAGExtractor
from src.graph_rag_store import GraphRAGStore

2025-06-26 22:33:16.994407: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750977197.013531    1784 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750977197.019859    1784 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-26 22:33:17.040775: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [0]:
# Manage environment variables required to attach LLM service in this context
exec(open('azure_envars.py').read())

In [0]:
from main import create_graph_index
save_idx_path = os.path.join(os.getcwd(), "outputs/palantir_8k_10k_graph")
save_kg_path = os.path.join(os.getcwd(), "outputs/8k_10k_kg.html")

index = create_graph_index(docs_dir= "data",
                           # num_nodes = 1,
                           save_index = True,
                           save_kg = True,
                           index_path = save_idx_path,
                           kg_path = save_kg_path
                        )
# index.property_graph_store.build_communities()

In [0]:
from src.helper_functions import kg_relations_to_df
kg_relations_to_df(index, save_as_csv = True, filename = "outputs/triplet_relations/8k_10k_kg_relations.csv")

,subject,predicate,object,filename,company,document_type,timestamp,period,relationship_description
0,Palantir Technologies Inc.,Issues,Class A Common Stock,2024 FY.pdf,Palantir Technologies,10K report,2024,Yearly,Palantir Technologies Inc. issues Class A Comm...
1,Class A Common Stock,Traded under symbol,PLTR,2024 FY.pdf,Palantir Technologies,10K report,2024,Yearly,Palantir's Class A Common Stock is traded unde...
2,PLTR,Listed on,The Nasdaq Stock Market LLC,2024 FY.pdf,Palantir Technologies,10K report,2024,Yearly,The PLTR ticker is listed on The Nasdaq Stock ...
3,Palantir Technologies Inc.,Files,10K report,2024 FY.pdf,Palantir Technologies,10K report,2024,Yearly,Palantir Technologies Inc. files an annual 10K...
4,10K report,Submitted to,United States Securities and Exchange Commission,2024 FY.pdf,Palantir Technologies,10K report,2024,Yearly,The 10K report is submitted to the SEC as requ...
...,...,...,...,...,...,...,...,...,...
4994,Twelfth Amendment,Amends,Credit Agreement,31_03_2022.pdf,Palantir Technologies,8K report,31-03-2022,,The Twelfth Amendment modifies the terms of th...
4995,Exhibit 10.1,Contains,Twelfth Amendment,31_03_2022.pdf,Palantir Technologies,8K report,31-03-2022,,Exhibit 10.1 includes the full text of the Twe...
4996,Palantir Technologies,No Outstanding Debt,Outstanding Amounts,31_03_2022.pdf,Palantir Technologies,8K report,31-03-2022,,"As of the report date, Palantir Technologies h..."
4997,Alexander C. Karp,Executive Officer,Palantir Technologies,31_03_2022.pdf,Palantir Technologies,8K report,31-03-2022,,Alexander C. Karp is the Chief Executive Offic...


In [0]:
from azure.identity import ClientSecretCredential, DefaultAzureCredential, get_bearer_token_provider  
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-large-en-v1.5")
llm = SafeAzureOpenAI(
    model= MODEL_NAME,
    deployment_name= DEPLOYMENT_NAME,
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    azure_ad_token_provider= token_provider,
    use_azure_ad= True,
    api_version=os.getenv("OPENAI_API_VERSION"),
    temperature=0,
)
Settings.embed_model = embed_model
Settings.llm = llm

In [0]:
from llama_index.core import load_index_from_storage, StorageContext
old_index = load_index_from_storage(storage_context=StorageContext.from_defaults(persist_dir="palantir_10k_graph"))

Loading llama_index.core.storage.kvstore.simple_kvstore from palantir_10k_graph/docstore.json.
Loading llama_index.core.storage.kvstore.simple_kvstore from palantir_10k_graph/index_store.json.


In [0]:
old_index = PropertyGraphIndex.from_existing(
    property_graph_store= index.property_graph_store  ,
    # optional, neo4j also supports vectors directly
    vector_store= index.vector_store,
    embed_kg_nodes=True,
)

In [0]:
from src.custom_query_engine import GraphRAGQueryEngine 

graph_store = index.property_graph_store
query_engine = GraphRAGQueryEngine(graph_store= graph_store,
                                   llm=llm,
                                   index=old_index
                                )
graph_store.build_communities()

query_str = "What was Palantir's revenue in 2022?"
response = query_engine.query(query_str)
response

In [0]:
nodes = index.as_retriever(
            similarity_top_k= 8
        ).retrieve(query_str)
nodes

[NodeWithScore(node=TextNode(id_='10940b15-1320-4346-9b28-5c4229112bb8', embedding=None, metadata={'company': 'Palantir Technologies', 'document_type': 'Financial report', 'year': '2022', 'period': 'FY'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='2022 FY', node_type='4', metadata={'company': 'Palantir Technologies', 'document_type': 'Financial report', 'year': '2022', 'period': 'FY'}, hash='0df669ed3f4792903f9dd2eec2a8d3218fcc6b2b19f26987e255d6f686545d74'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='aa5da375-da1d-4ec6-b91f-c8b8e803366b', node_type='1', metadata={'company': 'Palantir Technologies', 'document_type': 'Financial report', 'year': '2022', 'period': 'FY'}, hash='aaea765350213aaa14cd7a06c06d7c9a15547f29661bb43621b79e098caa6bcc'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='20ee1a95-4dd0-4bec-b521-ad203cb0594c', node_type='1', metadata={}, hash='f5564e6c8ac2a

[Trace(request_id=tr-e719d79487a5415aa31a3d3c3c5d3706), Trace(request_id=tr-5396ada6523e4d89a5a6fdeb68540ea5)]

In [0]:
class CustomRetriever(BaseRetriever):
    """Custom retriever that performs both semantic search and hybrid search."""

    def __init__(
        self,
        vector_retriever: VectorIndexRetriever,
        keyword_retriever: KeywordTableSimpleRetriever,
        mode: str = "AND",
    ) -> None:
        """Init params."""

        self._vector_retriever = vector_retriever
        self._keyword_retriever = keyword_retriever
        if mode not in ("AND", "OR"):
            raise ValueError("Invalid mode.")
        self._mode = mode
        super().__init__()

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """Retrieve nodes given query."""

        vector_nodes = self._vector_retriever.retrieve(query_bundle)
        keyword_nodes = self._keyword_retriever.retrieve(query_bundle)

        vector_ids = {n.node.node_id for n in vector_nodes}
        keyword_ids = {n.node.node_id for n in keyword_nodes}

        combined_dict = {n.node.node_id: n for n in vector_nodes}
        combined_dict.update({n.node.node_id: n for n in keyword_nodes})

        if self._mode == "AND":
            retrieve_ids = vector_ids.intersection(keyword_ids)
        else:
            retrieve_ids = vector_ids.union(keyword_ids)

        retrieve_nodes = [combined_dict[rid] for rid in retrieve_ids]
        return retrieve_nodes

In [0]:
from llama_index.core.indices.property_graph import LLMSynonymRetriever
from typing import List, Tuple

prompt = (
    "Given some initial query, generate synonyms or related keywords up to {max_keywords} in total, "
    "considering possible cases of capitalization, pluralization, common expressions, etc.\n"
    "Provide all synonyms/keywords separated by '^' symbols: 'keyword1^keyword2^...'\n"
    "Note, result should be in one-line, separated by '^' symbols."
    "----\n"
    "QUERY: {query_str}\n"
    "----\n"
    "KEYWORDS: "
)


def parse_fn(self, output: str) -> list[str]:
    matches = output.strip().split("^")

    # capitalize to normalize with ingestion
    return [x.strip().capitalize() for x in matches if x.strip()]

def parse_fn1(response_str: str) -> List[Tuple[str, str, str]]:
    lines = response_str.split("\n")
    triples = [line.split(",") for line in lines]
    return triples

synonym_retriever = LLMSynonymRetriever(
    index.property_graph_store,
    llm= llm,
    # include source chunk text with retrieved paths
    include_text=False,
    synonym_prompt=prompt,
    # output_parsing_fn= parse_fn1,
    max_keywords=10,
    # the depth of relations to follow after node retrieval
    path_depth=1,
)

retriever = index.as_retriever(sub_retrievers= [synonym_retriever])
retriever.retrieve(query_str)

## Neo4j Integration

In [0]:
from llama_index.graph_stores.neo4j import Neo4jPropertyGraphStore
from llama_index.core import PropertyGraphIndex

graph_store = Neo4jPropertyGraphStore(
    username="neo4j",
    password=" ",
    url=" ",
    )

In [0]:
from neo4j import GraphDatabase

uri = " "
username = "neo4j"
password = " "
driver = GraphDatabase.driver(uri, auth=(username, password))

def create_graph(tx, subject, predicate, object_):
    query = f"""
    MERGE (s:Entity {{name: $subject}})
    MERGE (o:Entity {{name: $object}})
    MERGE (s)-[r:{predicate.upper().replace(" ", "_")}]->(o)
    """
    tx.run(query, subject=subject, object=object_)

triplets = index.property_graph_store.get_triplets()
with driver.session() as session:
    for s, p, o in triplets:
        session.write_transaction(create_graph, s, p, o)
